In [142]:
import sys
import glob
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torch.utils.data
import numpy as np
import matplotlib.pylab as plt
import matplotlib.image as mpimg
import pandas as pd
from math import sin, cos
from PIL import ImageDraw, Image
import cv2
sys.path.append(os.path.abspath("./Residual-Attention-Network/"))
import seaborn as sns
import cv2
import time
import sklearn.preprocessing
from tqdm.notebook import tqdm
import torch.nn as nn
from sklearn.linear_model import LinearRegression
import torchvision
import paddle.fluid as paddle
import paddlehub as hub
from torchvision.models import resnet18
import sklearn.tree
import torch.nn.functional as F
module = hub.Module(name="faster_rcnn_coco2017")
%autosave 60

[2020-01-21 12:48:21,864] [    INFO] - Installing faster_rcnn_coco2017 module
[2020-01-21 12:48:21,879] [    INFO] - Module faster_rcnn_coco2017 already installed in /home/paperspace/.paddlehub/modules/faster_rcnn_coco2017


Autosaving every 60 seconds


## Faster RCNN stage

In [2]:
def pad_img(img, size = (112, 112, 3)):
    padded_img = np.zeros(size)
    pad_center_y = size[0]/2
    pad_center_x = size[1]/2
    if img.shape[0]>img.shape[1]:
        newwidth = int(img.shape[1]/img.shape[0]*size[0]//2*2)
        img = cv2.resize(img, (newwidth, size[0]))
        padded_img[:, int(pad_center_x-newwidth//2):int(pad_center_x+newwidth//2)] = img
    else:
        newheight = int(img.shape[0]/img.shape[1]*size[1]//2*2)
        img = cv2.resize(img, (size[1], newheight))
        padded_img[int(pad_center_y-newheight//2):int(pad_center_y+newheight//2), :] = img
    return padded_img
def iou(box1, box2):
    # each box is xmin, xmax, ymin, ymax
    xmin = max(box1[0], box2[0])
    xmax = min(box1[1], box2[1])
    ymin = max(box1[2], box2[2])
    ymax = min(box1[3], box2[3])
    if xmin>=xmax or ymin>=ymax:
        return 0
    box1area = (box1[1]-box1[0])*(box1[3]-box1[2])
    box2area = (box2[1]-box2[0])*(box2[3]-box2[2])
    intersection = (xmax-xmin)*(ymax-ymin)
    union = box1area+box2area-intersection
    return intersection/union
def find_boundingbox(img_dir, confidence_thres = 0.5):
    input_tensor = torch.zeros(1, 3, 2710, 3384).to('cuda')
    img = plt.imread(img_dir).astype('float32')/255
    input_tensor[0] = torch.Tensor(img.transpose((2, 0, 1)))
    output = model(input_tensor)[0]
    boxes = output['boxes']
    labels = output['labels']
    scores = output['scores']
    ego_box = [600, 3384, 2500, 2710]
    carboxes = []
    for box, label, confidence in zip(boxes, labels, scores):
        xmin, ymin, xmax, ymax = box
        box = [xmin, xmax, ymin, ymax]
        if label == 3 and confidence > confidence_thres:
            if iou(ego_box, box)==0:
                img_crop = img[int(ymin):int(ymax), int(xmin):int(xmax)]
                carboxes.append([img_crop, float(xmin), float(xmax), float(ymin), float(ymax), float(confidence)])
    return carboxes

In [3]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, box_nms_thresh = 0.3).to('cuda')
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d()
          )
  

In [36]:
test_filenames = glob.glob('./test_images/*.jpg')
#img_dir = test_filenames[np.random.randint(0, len(test_filenames))]
count = 0
df = pd.DataFrame(columns = ['original_name', 'pred_filename', 'ymin', 'ymax', 'xmin', 'xmax', 'confidence', 'overlap'])
for img_dir in tqdm(test_filenames):
    #begin = time.time()
    boxes = find_boundingbox(img_dir, 0.5)
    #print((time.time()-begin)/1, 'seconds')

    predboxes = []
    mask_dir = img_dir.replace('test_images', 'test_masks')
    if len(glob.glob(mask_dir))>0:
        ignore_mask = plt.imread(mask_dir).mean(axis = 2)/255
    else:
        ignore_mask = np.zeros((3384, 2710, 3))
    #plt.imshow(ignore_mask)
    #plt.show()
    for predbox in boxes:
        img_crop = predbox[0]
        xmin, xmax, ymin, ymax = predbox[1:-1]
        if np.sum(ignore_mask[int(ymin):int(ymax), int(xmin):int(xmax)])>(int(xmax)-int(xmin))*(int(ymax)-int(ymin))*0.25:
            pass
            #print('overlap with ignore mask', np.sum(ignore_mask[int(ymin):int(ymax), int(xmin):int(xmax)])/((int(xmax)-int(xmin))*(int(ymax)-int(ymin))))
        else:
            predboxes.append(predbox)

    #print(truthboxes)

    print(len(predboxes), 'cars detected')
    for i, box1 in enumerate(predboxes):
        crop_pred, box1, confidence = box1[0], box1[1:-1], box1[-1]
        #plt.imshow(crop_pred)
        #plt.show()
        #print('confidence:', confidence)
            # starting writing into csv
        row = {}
        pred_filename = './test_results/img_crops/'+"%05d"%count+'.jpg'
        row['xmin'], row['xmax'], row['ymin'], row['ymax'] = box1
        row['original_name'], row['pred_filename'], row['confidence'] = img_dir, pred_filename, confidence
        #check ignore mask overlap
        xmin, xmax, ymin, ymax = box1
        overlap = np.sum(ignore_mask[int(ymin):int(ymax), int(xmin):int(xmax)])/(int(xmax)-int(xmin))/(int(ymax)-int(ymin))
        #print('ignore mask overlap', overlap)
        row['overlap'] = overlap
        plt.imsave(pred_filename, crop_pred)
        df = df.append(row, ignore_index = True)
        count+=1
        #end writing into csv
        ### do ensembling on this later
    #original_img = plt.imread(img_dir)
    #plt.figure(figsize = (30, 20))
    #plt.imshow(original_img)
    #plt.show()
df.to_csv('./test_results/rcnn_preds.csv')




7 cars detected
10 cars detected
11 cars detected
15 cars detected
11 cars detected
10 cars detected
3 cars detected
11 cars detected
4 cars detected
26 cars detected
7 cars detected
8 cars detected
7 cars detected
11 cars detected
6 cars detected
14 cars detected
9 cars detected
5 cars detected
15 cars detected
8 cars detected
10 cars detected
20 cars detected
13 cars detected
9 cars detected
15 cars detected
9 cars detected
8 cars detected
18 cars detected
13 cars detected
23 cars detected
12 cars detected
7 cars detected
13 cars detected
21 cars detected
4 cars detected
11 cars detected
19 cars detected
14 cars detected
9 cars detected
11 cars detected
8 cars detected
8 cars detected
15 cars detected
5 cars detected
6 cars detected
21 cars detected
5 cars detected
10 cars detected
9 cars detected
10 cars detected
20 cars detected
9 cars detected
11 cars detected
14 cars detected
18 cars detected
4 cars detected
12 cars detected
6 cars detected
6 cars detected
9 cars detected
15 cars

4 cars detected
16 cars detected
8 cars detected
13 cars detected
16 cars detected
7 cars detected
10 cars detected
11 cars detected
9 cars detected
11 cars detected
9 cars detected
11 cars detected
12 cars detected
13 cars detected
20 cars detected
9 cars detected
16 cars detected
8 cars detected
16 cars detected
6 cars detected
4 cars detected
7 cars detected
3 cars detected
15 cars detected
12 cars detected
9 cars detected
21 cars detected
8 cars detected
16 cars detected
12 cars detected
9 cars detected
15 cars detected
20 cars detected
8 cars detected
16 cars detected
14 cars detected
13 cars detected
15 cars detected
8 cars detected
15 cars detected
17 cars detected
21 cars detected
14 cars detected
10 cars detected
11 cars detected
7 cars detected
23 cars detected
14 cars detected
6 cars detected
9 cars detected
12 cars detected
5 cars detected
5 cars detected
9 cars detected
5 cars detected
24 cars detected
14 cars detected
5 cars detected
12 cars detected
8 cars detected
9 car

5 cars detected
12 cars detected
12 cars detected
11 cars detected
16 cars detected
13 cars detected
14 cars detected
18 cars detected
17 cars detected
10 cars detected
12 cars detected
15 cars detected
12 cars detected
20 cars detected
9 cars detected
23 cars detected
17 cars detected
8 cars detected
9 cars detected
15 cars detected
23 cars detected
11 cars detected
13 cars detected
11 cars detected
7 cars detected
19 cars detected
8 cars detected
13 cars detected
8 cars detected
24 cars detected
7 cars detected
13 cars detected
13 cars detected
16 cars detected
13 cars detected
5 cars detected
12 cars detected
15 cars detected
11 cars detected
5 cars detected
10 cars detected
16 cars detected
11 cars detected
12 cars detected
3 cars detected
13 cars detected
20 cars detected
7 cars detected
5 cars detected
12 cars detected
14 cars detected
6 cars detected
14 cars detected
14 cars detected
13 cars detected
12 cars detected
15 cars detected
8 cars detected
18 cars detected
8 cars detec

10 cars detected
6 cars detected
7 cars detected
5 cars detected
10 cars detected
15 cars detected
11 cars detected
12 cars detected
24 cars detected
9 cars detected
20 cars detected
8 cars detected
8 cars detected
9 cars detected
12 cars detected
17 cars detected
15 cars detected
14 cars detected
10 cars detected
10 cars detected
12 cars detected
13 cars detected
6 cars detected
12 cars detected
7 cars detected
11 cars detected
10 cars detected
13 cars detected
5 cars detected
19 cars detected
15 cars detected
25 cars detected
6 cars detected
15 cars detected
26 cars detected
10 cars detected
5 cars detected
12 cars detected
12 cars detected
17 cars detected
6 cars detected
15 cars detected
11 cars detected
15 cars detected
14 cars detected
9 cars detected
5 cars detected
17 cars detected
7 cars detected
19 cars detected
9 cars detected
9 cars detected
14 cars detected
14 cars detected
13 cars detected
21 cars detected
8 cars detected
14 cars detected
9 cars detected
10 cars detected


16 cars detected
11 cars detected
6 cars detected
16 cars detected
5 cars detected
12 cars detected
11 cars detected
18 cars detected
12 cars detected
4 cars detected
8 cars detected
6 cars detected
15 cars detected
13 cars detected
6 cars detected
13 cars detected
15 cars detected
15 cars detected
9 cars detected
8 cars detected
16 cars detected
11 cars detected
9 cars detected
10 cars detected
6 cars detected
19 cars detected
5 cars detected
13 cars detected
8 cars detected
15 cars detected
19 cars detected
14 cars detected
8 cars detected
18 cars detected
9 cars detected
15 cars detected
9 cars detected
19 cars detected
19 cars detected
10 cars detected
15 cars detected
5 cars detected
12 cars detected
19 cars detected
25 cars detected
11 cars detected
12 cars detected
30 cars detected
10 cars detected
16 cars detected
9 cars detected
6 cars detected
5 cars detected
12 cars detected
16 cars detected



In [31]:
df

,Unnamed: 0,original_name,pred_filename,ymin,ymax,xmin,xmax,confidence,overlap
0,0,./test_images/ID_b2d1b80e3.jpg,./test_results00000.jpg,1794.820923,2026.436279,1542.635132,1823.220215,0.999056,0.000000
1,1,./test_images/ID_b2d1b80e3.jpg,./test_results00001.jpg,1708.076904,1820.164062,2251.672607,2434.299072,0.990160,0.000000
2,2,./test_images/ID_b2d1b80e3.jpg,./test_results00002.jpg,1840.223022,1986.309814,7.472916,275.578888,0.965210,0.000000
3,3,./test_images/ID_b2d1b80e3.jpg,./test_results00003.jpg,1777.652710,1864.642578,676.919861,878.848450,0.911132,0.000000
4,4,./test_images/ID_b2d1b80e3.jpg,./test_results00004.jpg,1631.602539,1740.061035,3184.773438,3380.093506,0.897868,0.000000
5,5,./test_images/ID_b2d1b80e3.jpg,./test_results00005.jpg,1731.692993,1774.867432,915.703003,973.209900,0.822143,0.000000
6,6,./test_images/ID_b2d1b80e3.jpg,./test_results00006.jpg,1737.764771,1846.164795,116.846542,364.030060,0.640237,0.000000
7,7,./test_images/ID_141add734.jpg,./test_results00007.jpg,1856.315308,2286.692871,439.957947,1157.179077,0.998335,0.000000
8,8,./test_images/ID_141add734.jpg,./test_results00008.jpg,1816.647583,2003.348389,1942.202393,2179.670898,0.995073,0.000000
9,9,./test_images/ID_141add734.jpg,./test_results00009.jpg,1919.364258,2336.636230,2175.693359,2747.678223,0.994852,0.000000


In [22]:
import torch
import torch.nn as nn
from torch.nn import init
import functools
from torch.autograd import Variable
import numpy as np
from model.basic_layers import ResidualBlock
from model.attention_module import AttentionModule_stage1, AttentionModule_stage2, AttentionModule_stage3, AttentionModule_stage0
from model.attention_module import AttentionModule_stage1_cifar, AttentionModule_stage2_cifar, AttentionModule_stage3_cifar

In [23]:
# def get_mesh(batch_size, shape_y, shape_x, box_specs):
#     mesh = torch.zeros([batch_size, 2, shape_y, shape_x])
#     for i in range(batch_size):
#         xmax, ymax, xmin, ymin = box_specs[i].detach().numpy()
#         mg_y, mg_x = np.meshgrid(np.linspace(ymin, ymax, shape_y), np.linspace(xmin, xmax, shape_x), indexing = 'ij')
#         mesh[i, 0, :, :] = torch.Tensor(mg_y)
#         mesh[i, 1, :, :] = torch.Tensor(mg_x)
#     return mesh

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=3):
        self.inplanes = 64 #64
        super(ResNet, self).__init__()
#        self.simplemodel = reallysimplemodel()
        self.conv1 = nn.Conv2d(5, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn0 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.selu = nn.SELU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0]) #64
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2) #128
        self.attention_module3 = AttentionModule_stage3(128, 128)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2) #256
        self.attention_module4 = AttentionModule_stage3(256, 256, (7, 7))
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2) #512
        self.avgpool = nn.AvgPool2d(4, stride=1)
        self.fc = nn.Linear(8192 * block.expansion, 200)
        self.fc1 = nn.Linear(200, 6)
        #self.bn1 = nn.BatchNorm1d(300)
#         self.fc2 = nn.Linear(300, 500)
#         self.bn2 = nn.BatchNorm1d(500)
#         self.fc3 = nn.Linear(500, 300)
#         self.bn3 = nn.BatchNorm1d(300)
#         self.fc4 = nn.Linear(300, 200)
#         self.bn4 = nn.BatchNorm1d(200)
#         self.fc5 = nn.Linear(200, 200)
#         self.bn5 = nn.BatchNorm1d(200)
        #self.fc6 = nn.Linear(300, 3)
        self.relu = nn.ReLU()
        self.selu = nn.SELU()
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        ## to compensate for the meshgrid for next round
        #self.inplanes += 2
        return nn.Sequential(*layers)
    def forward(self, x):
        x = self.conv1(x)    # 224x224
        x = self.bn0(x)     
        x = self.relu(x)
        x = self.maxpool(x)  # 112x112
        x = self.layer1(x)   # 56x56     
        x = self.layer2(x)   # 28x28
        x = self.attention_module3(x)
        x = self.layer3(x)   # 14x14
        x = self.attention_module4(x)
        x = self.layer4(x)   # 7x7
        x = x.view(x.size(0), -1)
        X = self.relu(self.fc(x))
        X = self.fc1(X)
        return X

In [129]:
from efficientnet_pytorch import EfficientNet
class BackBone(nn.Module):
    def __init__(self, block, layers, num_classes=3):
        self.inplanes = 64 #64
        super(BackBone, self).__init__()
#        self.simplemodel = reallysimplemodel()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn0 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.selu = nn.SELU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0]) #64
        self.layer2 = self._make_layer(block, 256, layers[1], stride=2) #128
        self.attention_module3 = AttentionModule_stage3(256, 256, (16, 42))
        self.layer3 = self._make_layer(block, 512, layers[2], stride=2) #256
        self.attention_module4 = AttentionModule_stage3(512, 512, (8, 21))
        self.layer4 = self._make_layer(block, 1024, layers[3], stride=2) #512

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)
    def forward(self, x):
        x = self.conv1(x)    # 224x224
        x = self.bn0(x)     
        x = self.relu(x)
        x = self.maxpool(x)  # 112x112   
        x = self.layer1(x)   # 56x56
        x = self.layer2(x)   # 28x28
        x = self.attention_module3(x)
        x = self.layer3(x)   # 14x14
        x = self.attention_module4(x)
        x = self.layer4(x)   # 7x7
        return x
    
    
class double_conv(nn.Module):
    '''(conv => BN => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.conv(x)
        return x

class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(up, self).__init__()

        #  would be a nice idea if the upsampling could be learned too,
        #  but my machine do not have enough memory to handle all those weights
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch//2, in_ch//2, 2, stride=2)

        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x1, x2=None):
        x1 = self.up(x1)
        
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, (diffX // 2, diffX - diffX//2,
                        diffY // 2, diffY - diffY//2))
        
        # for padding issues, see 
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        
        if x2 is not None:
            x = torch.cat([x2, x1], dim=1)
        else:
            x = x1
        x = self.conv(x)
        return x

def get_mesh(batch_size, shape_x, shape_y):
    mg_x, mg_y = np.meshgrid(np.linspace(0, 1, shape_y), np.linspace(0, 1, shape_x))
    mg_x = np.tile(mg_x[None, None, :, :], [batch_size, 1, 1, 1]).astype('float32')
    mg_y = np.tile(mg_y[None, None, :, :], [batch_size, 1, 1, 1]).astype('float32')
    mesh = torch.cat([torch.tensor(mg_x).to('cuda'), torch.tensor(mg_y).to('cuda')], 1)
    return mesh

In [137]:
class MyUNet(nn.Module):
    '''Mixture of previous classes'''
    def __init__(self, n_classes):
        super(MyUNet, self).__init__()
        self.base_model = net = BackBone(BasicBlock, [3, 4, 6, 3]).to('cuda')
        
        self.conv0 = double_conv(6, 64)
        self.conv1 = double_conv(64, 128)
        self.conv2 = double_conv(128, 512)
        self.conv3 = double_conv(512, 1024)
        
        self.mp = nn.MaxPool2d(2)
        
        self.up1 = up(1024 + 1024 + 2, 512)
        self.up2 = up(512 + 512, 256)
        self.up3 = up(256+128, 128)
        self.up4 = up(128+64, 64)
        self.up5 = up(64+6, 64)
        self.outc = nn.Conv2d(64, n_classes, 1)
 
    def forward(self, x):
        masks = x[:, 3:4, :, :]
        batch_size = x.shape[0]
        mesh1 = get_mesh(batch_size, x.shape[2], x.shape[3])
        x0 = torch.cat([x, mesh1], 1)
        x1 = self.mp(self.conv0(x0))
        x2 = self.mp(self.conv1(x1))
        x3 = self.mp(self.conv2(x2))
        x4 = self.mp(self.conv3(x3))
        
        x_center = x[:, :3, :, IMG_WIDTH // 8: -IMG_WIDTH // 8]
        feats = self.base_model(x_center)
        bg = torch.zeros([feats.shape[0], feats.shape[1], feats.shape[2], feats.shape[3] // 8]).to('cuda')
        feats = torch.cat([bg, feats, bg], 3)
        
        # Add positional info
        mesh2 = get_mesh(batch_size, feats.shape[2], feats.shape[3])
        feats = torch.cat([feats, mesh2], 1)
        x = self.up1(feats, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.up5(x, x0)
        x = self.outc(x)
        return x

In [138]:
leftnet = ResNet(BasicBlock, [2, 2, 2, 2]).to('cuda')
leftnet.load_state_dict(torch.load('./Joseph_Dataset/weights/leftnet.pth'))
leftnet.eval()
rightnet = ResNet(BasicBlock, [2, 2, 2, 2]).to('cuda') 
rightnet.load_state_dict(torch.load('./Joseph_Dataset/weights/rightnet.pth'))
rightnet.eval()
xyzmodel = MyUNet(3).to('cuda')
pass

In [139]:
def rotate(x, angle):
    x = x + angle
    x = x - (x + np.pi) // (2 * np.pi) * 2 * np.pi
    return x
def pad_img(img, size = (112, 112, 3)):
    padded_img = np.zeros(size)
    pad_center_y = size[0]/2
    pad_center_x = size[1]/2
    if img.shape[0]>img.shape[1]:
        newwidth = int(img.shape[1]/img.shape[0]*size[0]//2*2)
        img = cv2.resize(img, (newwidth, size[0]))
        padded_img[:, int(pad_center_x-newwidth//2):int(pad_center_x+newwidth//2)] = img
    else:
        newheight = int(img.shape[0]/img.shape[1]*size[1]//2*2)
        img = cv2.resize(img, (size[1], newheight))
        padded_img[int(pad_center_y-newheight//2):int(pad_center_y+newheight//2), :] = img
    return padded_img
def get_numpy_mesh(shape_y, shape_x, box):
    mesh = np.zeros([shape_y, shape_x, 2])
    
    xmax, ymax, xmin, ymin = box
    mg_y, mg_x = np.meshgrid(np.linspace(ymin, ymax, shape_y), np.linspace(xmin, xmax, shape_x), indexing = 'ij')
    mesh[:, :, 0] = mg_y
    mesh[:, :, 1] = mg_x
    return mesh.astype('float32')

In [140]:
def calculate_angle_back(yaw_cos, yaw_sin, pitch, roll):
    if yaw_cos<0: ## if np.cos(yaw)>0 is False
        yaw_sin = yaw_sin*-1
    yaw = np.arcsin(yaw_sin)
    if yaw_cos<0:
        yaw = rotate(yaw, np.pi)
    roll = rotate(roll, np.pi)
    return yaw, pitch, roll
def calculate_centers_back(xmin, xmax, ymin, ymax, xcenter, ycenter):
    longerside = max(xmax-xmin, ymax-ymin)
    xmiddle = (xmax+xmin)/2
    ymiddle = (ymax+ymin)/2
    xcenter = xmiddle+longerside/2*xcenter
    ycenter = ymiddle+longerside/2*ycenter
    return xcenter, ycenter

k = np.array([[2304.5479, 0,  1686.2379],
                          [0, 2305.8757, 1354.9849],
                          [0, 0, 1]], dtype=np.float32)
IMG_WIDTH = 448 #after padding
IMG_HEIGHT = 128 #after cutoff
padding_x = 3384//6
def make_input_and_mask(filename, xcenters, ycenters, yaws, pitches, rolls, shape_y = IMG_HEIGHT, shape_x = IMG_WIDTH, cutoff_y = 1600):
    img = np.zeros((2710, 3384+padding_x*2, 3), dtype = 'float32')
    img[:, padding_x:-padding_x, :] = plt.imread(filename).astype('float32')/255
    mask = np.zeros((img.shape[0], img.shape[1], 1), dtype = 'float32') ## center mask
    mask_angle = np.zeros((img.shape[0], img.shape[1], 3), dtype = 'float32') ## angle mask
    # resize everything
    input_tensor = np.concatenate([img, mask], axis = 2)
    input_tensor = input_tensor[cutoff_y:]
    input_tensor = cv2.resize(input_tensor, (shape_x, shape_y))
    out_mask = cv2.resize(input_tensor[:, :, 3], (shape_x, shape_y))[:, :, np.newaxis]
    mask_angle = cv2.resize(mask_angle, (shape_x, shape_y))
    ## fill in the masks
    for xcenter, ycenter, yaw, pitch, roll in zip(xcenters, ycenters, yaws, pitches, rolls):
        if xcenter<-padding_x or xcenter>3384+padding_x or ycenter<0 or ycenter>2710:
            print('fuck')# if result is too ridiculous
            continue
        ## transform the centers for resizing
        xcenter = (xcenter+padding_x)/(3384+padding_x*2)*shape_x
        ycenter = (ycenter-cutoff_y)/(2710-cutoff_y)*shape_y
        input_tensor[int(ycenter), int(xcenter), 3] = 1
        out_mask[int(ycenter), int(xcenter)] = 1
        mask_angle[int(ycenter), int(xcenter)] = [yaw, pitch, roll]
    return input_tensor.astype('float32'), mask_angle.astype('float32'), out_mask.astype('float32')

def transform_xyz_back(img_x, img_y, z): ### optimize xyz based on img_x, img_y, and z
    img_x = img_x/IMG_WIDTH*(3384+3384//6*2)-3384//6
    img_y = img_y/IMG_HEIGHT*(2710-1600)+1600
    camera_x = z*img_x
    camera_y = z*img_y
    x, y, z = np.dot(np.linalg.inv(k), [camera_x, camera_y, z])
    return x, y, z


def project(x, y, z):   ## turn xyz into img_x, img_y
    camera_x, camera_y, camera_z = np.dot(k, [x, y, z])
    return camera_x/camera_z, camera_y/camera_z


In [149]:
df = pd.read_csv('./test_results/rcnn_preds.csv')  
# interate over photos
for img_dir in tqdm(test_filenames[:60]):
    xcenters, ycenters, yaws, pitches, rolls = [], [], [], [], []
    df_img = df[df['original_name'] == img_dir]
    # iterate over crops
    for idx in range(len(df_img)):
        dic = dict(df_img.iloc[idx])
        keys = ['pred_filename','xmin', 'xmax', 'ymin', 'ymax']
        pred_filename, xmin, xmax, ymin, ymax = [dic[key] for key in keys]
        # make the input
        img = plt.imread(pred_filename).astype('float32')/255
        height, width, channels = img.shape
        mesh = get_numpy_mesh(height, width,[xmax/3384, ymax/2710, xmin/3384, ymin/2710])
        # img_copy = img.copy()
        img = np.concatenate([img, mesh], axis = 2)
        img = pad_img(img, size = (112, 112, 5))
        img = img.transpose((2, 0, 1))
        xmiddle = (xmax+xmin)/2
        ymiddle = (ymax+ymin)/2
        img = torch.Tensor(img[np.newaxis, :, :, :]).to('cuda')
        # select which neural network to use
        if xmiddle>3384/2:
            outputs = rightnet(img)
        else:
            outputs = leftnet(img)
        outputs = outputs.cpu().detach().numpy()[0]
        # de-normalize the regression results
        yaw, pitch, roll = calculate_angle_back(*outputs[:4])
        # recover screwed up order
        pitch, yaw, roll = -yaw, -pitch, -roll
        xcenter, ycenter = calculate_centers_back(xmin, xmax, ymin, ymax, outputs[4], outputs[5])
        # make a list for each image
        xcenters.append(xcenter)
        ycenters.append(ycenter)
        yaws.append(yaw)
        pitches.append(pitch)
        rolls.append(roll)
        # uncomment to visualize
#         if np.cos(yaw)>0:
#             print('looking away')
#         else:
#             print('looking towards you')
        #img_copy = cv2.circle(img_copy, (int(xcenter-xmin), int(ycenter-ymin)), 3, (0, 1, 0), -1)
        #plt.imshow(img_copy)
        #plt.show()
    input_tensor, mask_angle, out_mask = make_input_and_mask(img_dir, xcenters, ycenters, yaws, pitches, rolls)
    input_tensor = torch.Tensor(input_tensor.transpose((2, 0, 1))[np.newaxis, ...]).cuda()
    output = xyzmodel(input_tensor)
    output = output[0].cpu().detach().numpy().transpose((1, 2, 0))*out_mask
    img = plt.imread(img_dir)
    #draw the points
    for i in range(out_mask.shape[0]):
        for j in range(out_mask.shape[1]):
            if out_mask[i][j]!=0:
                x, y, z = output[i][j]*100
                x, y, z = transform_xyz_back(j, i, z) ## optimize projection
                img_x, img_y = project(x, y, z)
                img = cv2.circle(img, (int(img_x), int(img_y)), int(100/z), tuple(mask_angle[i,j]), -1)
    plt.figure(figsize = (30, 20))
    plt.imshow(img.get())
    

(0.12152797, -0.098510645, -3.1201348)


TypeError: Scalar value for argument 'color' is not numeric